In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from torchvision import transforms
from torch.utils.data import Dataset
from PIL import Image
import os

In [2]:
import os
import torch
import timm
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch.optim import Adam
from sklearn.metrics import accuracy_score


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

### Data Preparation

In [5]:
class GenderDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.samples = []
        self.transform = transform
        for label, gender in enumerate(["Male", "Female"]):
            folder = os.path.join(root_dir, gender)
            for img in os.listdir(folder):
                path = os.path.join(folder, img)
                self.samples.append((path, label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        image = Image.open(path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label

In [6]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

In [7]:
train_dataset = GenderDataset("/kaggle/input/2425-ii-ait-3002-1-gender-classification/Dataset/Train", transform=train_transform)
val_dataset = GenderDataset("/kaggle/input/2425-ii-ait-3002-1-gender-classification/Dataset/Val", transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

### Model Definition

In [8]:
model = timm.create_model("convnext_base", pretrained=True, num_classes=2)
model.to(device)

model.safetensors:   0%|          | 0.00/354M [00:00<?, ?B/s]

ConvNeXt(
  (stem): Sequential(
    (0): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
    (1): LayerNorm2d((128,), eps=1e-06, elementwise_affine=True)
  )
  (stages): Sequential(
    (0): ConvNeXtStage(
      (downsample): Identity()
      (blocks): Sequential(
        (0): ConvNeXtBlock(
          (conv_dw): Conv2d(128, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=128)
          (norm): LayerNorm((128,), eps=1e-06, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=128, out_features=512, bias=True)
            (act): GELU()
            (drop1): Dropout(p=0.0, inplace=False)
            (norm): Identity()
            (fc2): Linear(in_features=512, out_features=128, bias=True)
            (drop2): Dropout(p=0.0, inplace=False)
          )
          (shortcut): Identity()
          (drop_path): Identity()
        )
        (1): ConvNeXtBlock(
          (conv_dw): Conv2d(128, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), g

### Train

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=1e-4)

In [11]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
scheduler = ReduceLROnPlateau(optimizer, mode='max', patience=2, factor=0.5, verbose=True)

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [12]:
best_acc = 0
early_stop_counter = 0
patience = 5

for epoch in range(30): 
    model.train()
    total_loss, total_correct, total_samples = 0.0, 0, 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * images.size(0)
        preds = torch.argmax(outputs, dim=1)
        total_correct += (preds == labels).sum().item()
        total_samples += images.size(0)

    train_loss = total_loss / total_samples
    train_acc = total_correct / total_samples

    # Validation
    model.eval()
    val_preds, val_labels = [], []
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1).cpu()
            val_preds.extend(preds)
            val_labels.extend(labels)

    val_acc = accuracy_score(val_labels, val_preds)
    scheduler.step(val_acc)  

    print(f"Epoch {epoch+1}: Train Loss = {train_loss:.4f}, "
          f"Train Acc = {train_acc:.4f}, Val Acc = {val_acc:.4f}")

    # Early stopping logic
    if val_acc > best_acc:
        best_acc = val_acc
        early_stop_counter = 0
        torch.save(model.state_dict(), "best_model.pth")
    else:
        early_stop_counter += 1
        if early_stop_counter >= patience:
            print("Early stopping triggered.")
            break

Epoch 1: Train Loss = 0.0688, Train Acc = 0.9778, Val Acc = 0.9880
Epoch 2: Train Loss = 0.0248, Train Acc = 0.9912, Val Acc = 0.9890
Epoch 3: Train Loss = 0.0142, Train Acc = 0.9958, Val Acc = 0.9810
Epoch 4: Train Loss = 0.0146, Train Acc = 0.9955, Val Acc = 0.9800
Epoch 5: Train Loss = 0.0071, Train Acc = 0.9975, Val Acc = 0.9810
Epoch 6: Train Loss = 0.0019, Train Acc = 0.9995, Val Acc = 0.9840
Epoch 7: Train Loss = 0.0001, Train Acc = 1.0000, Val Acc = 0.9860
Early stopping triggered.


### Inference + TTA + Submission

In [13]:
# Load best model
model.load_state_dict(torch.load("best_model.pth"))
model.eval()

ConvNeXt(
  (stem): Sequential(
    (0): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
    (1): LayerNorm2d((128,), eps=1e-06, elementwise_affine=True)
  )
  (stages): Sequential(
    (0): ConvNeXtStage(
      (downsample): Identity()
      (blocks): Sequential(
        (0): ConvNeXtBlock(
          (conv_dw): Conv2d(128, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=128)
          (norm): LayerNorm((128,), eps=1e-06, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=128, out_features=512, bias=True)
            (act): GELU()
            (drop1): Dropout(p=0.0, inplace=False)
            (norm): Identity()
            (fc2): Linear(in_features=512, out_features=128, bias=True)
            (drop2): Dropout(p=0.0, inplace=False)
          )
          (shortcut): Identity()
          (drop_path): Identity()
        )
        (1): ConvNeXtBlock(
          (conv_dw): Conv2d(128, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), g

In [14]:
tta_transforms = [
    transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.5]*3, [0.5]*3)
    ]),
    transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(p=1.0),
        transforms.ToTensor(),
        transforms.Normalize([0.5]*3, [0.5]*3)
    ])
]

In [15]:
test_folder = "/kaggle/input/2425-ii-ait-3002-1-gender-classification/Dataset/Test"
results = []

with torch.no_grad():
    for img_name in tqdm(sorted(os.listdir(test_folder))):
        path = os.path.join(test_folder, img_name)
        image = Image.open(path).convert("RGB")
        preds = []
        for t in tta_transforms:
            img = t(image).unsqueeze(0).to(device)
            out = model(img)
            preds.append(torch.softmax(out, dim=1))
        avg_pred = torch.mean(torch.stack(preds), dim=0)
        label = torch.argmax(avg_pred, dim=1).item()
        results.append((img_name.split('.')[0], label))

100%|██████████| 1000/1000 [00:37<00:00, 26.55it/s]


In [16]:
df = pd.DataFrame(results, columns=["ID", "Label"])
df.to_csv("submission.csv", index=False)
